In [1]:
%matplotlib inline
import xarray as xr
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import netCDF4
from scipy import stats
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import sys
from mpl_toolkits.axes_grid1 import make_axes_locatable
sys.path.append('/Users/gbromley/code/python_utilities/')
from python_data_functions import extract_months
from python_met_functions import sat_vap_pres
import metpy.calc as mcalc
from metpy.units import units

In [2]:
data_dir = '/Users/gbromley/data/CRU/'
temp_file = 'tmp/cru_ts3.24.1901.2015.tmp.dat.nc'
#vap_file = 'vap/cru_ts3.24.1901.2015.vap.dat.nc'
#nc_vap = xr.open_dataset(data_dir+vap_file)
nc_temp = xr.open_dataset(data_dir+temp_file)

In [3]:
t2m = nc_temp['tmp']


In [4]:
t_season='MJJAS'
t_months=[5,6,7,8,9]
start_year = '1970-01-01'
end_year = '2010-12-01'
ext_e = -100
ext_w = -115
ext_n = 50
ext_s = 42

ggw_lat,ggw_lon = 48.18, -106.635

In [5]:
#extract the time period we are interested in
t2m_slice=t2m.sel(time=slice(start_year,end_year))
#grab the months interested in
t2m_months = t2m_slice.sel(time=extract_months(t2m_slice['time.month'],t_months[0],t_months[-1]))
#create month averages
t2m_months_avg = t2m_months.groupby('time.year').mean(dim='time')

#extract the 30 yr climate normal times
t2m_clim_30= t2m.sel(time=slice('1981-01-01','2010-12-01'))
#grab the months
t2m_clim_months_30 = t2m_clim_30.sel(time=extract_months(t2m_clim_30['time.month'],t_months[0],t_months[-1]))
#create averages over months for climate normal
t2m_clim_months_avg = t2m_clim_months_30.mean(dim='time')

In [6]:
#create 2d structure to hold slope
spatial_trend = t2m.isel(time=1).copy(deep=True)
spatial_trend.name = 't2m_trend'
spatial_trend.attrs['units'] = 'C per Decade'
spatial_trend.attrs['long_name'] = '2 meter temperature trend'

#create 2d structure to hold pvalues
pvalues= t2m.isel(time=1).copy(deep=True)
pvalues.name = 't2m_trend_pvalues'
pvalues.attrs['units'] = 'pvalues'
pvalues.attrs['long_name'] = '2 meter temperature trend pvalues'

In [7]:
#calculate slope and pvalue for each grid point
#replace with a map() call?
for i in np.arange(0,len(t2m['lat'])):
    for j in np.arange(0,len(t2m['lon'])):
        series = t2m_months_avg[:,i,j]
        anom = series-t2m_clim_months_avg[i,j]
        slope, intercept, r_value, p_value, std_err = stats.linregress(np.arange(0,len(anom)),anom)
        spatial_trend[i,j]=slope*10
        pvalues[i,j]=p_value
        #print(i)

In [8]:
spatial_trend.to_dataset().to_netcdf(data_dir+'CRU_temp_tobias'+t_season+'_'+'1970'+'.nc')

In [9]:
spatial_trend.max()


<xarray.DataArray 't2m_trend' ()>
array(2.533240418118467)
Coordinates:
    time     datetime64[ns] 1901-02-15

In [10]:
anom_series = t2m_months_avg - t2m_clim_months_avg

In [11]:
test_series = anom_series.sel(lat=48.75)
test_series = test_series.sel(lon=-106.75)
anom_series = test_series
slope, intercept, r_value, p_value, std_err = stats.linregress(np.arange(0,len(anom_series)),anom_series.values)
line = slope*np.arange(0,len(anom_series.values))+intercept
anom_series = anom_series.to_pandas()

In [ ]:
fig = plt.figure(figsize=(13,7), dpi=1200.0)
projection = ccrs.AlbersEqualArea(central_longitude=-111.0,central_latitude=46.0,false_easting=0.0, false_northing=0.0, standard_parallels=(40.0, 50.0), globe=None)
ax1 = plt.subplot(1,1,1, projection = projection)
#ax1 = plt.axes(projection=ccrs.AlbersEqualArea(central_longitude=-111.0,central_latitude=46.0,false_easting=0.0, false_northing=0.0, standard_parallels=(40.0, 50.0), globe=None))
ax1.set_global()
test = spatial_trend.plot.pcolormesh(axes = ax1, transform=ccrs.PlateCarree(),vmin=-0.0013, vmax=0.0013, cmap='RdBu_r',add_colorbar = False)
#ax = plt.axes(projection=ccrs.LambertConformal())

ax1.set_extent([ext_e, ext_w, ext_s, ext_n])
states_provinces = cfeature.NaturalEarthFeature(
    category='cultural',
    name='admin_1_states_provinces_lines',
    scale='50m',
    facecolor='none')
ax1.add_feature(states_provinces, edgecolor='gray')
ax1.add_feature(cfeature.COASTLINE)
ax1.add_feature(cfeature.BORDERS)
plt.plot(ggw_lon, ggw_lat, axes = ax1, marker='o', markersize = 8, color='black', transform=ccrs.PlateCarree())
plt.text(ggw_lon-2, ggw_lat, 'GGW', axes=ax1, color='black', size=8, transform=ccrs.PlateCarree())
ax1.title.set_visible(False)
cb = plt.colorbar(test, cmap='RdBu_r',fraction=0.035)
cb.set_label('Kg/Kg / Decade',fontsize=10)
cb.ax.tick_params(labelsize=8)



#ax2 = plt.subplot(1,2,2)
#anom_series.rolling(window=5).mean().plot(ax=ax2);
#anom_series.plot(ax=ax2)
#trend = ax.plot(anom_series['time'],line)
#base = np.zeros(len(anom_series.values))
#ax2.plot(anom_series.index.values,base,c='grey')
#ax2.set_title('Temp at GGW'+start_year[0:4]+'-'+end_year[0:4]+' '+t_season)
#ax2.set_xlabel('Year')
#ax2.set_ylabel('Temperature Anomaly [$^\circ$C]')
#trend = ax2.plot(anom_series.index.values,line)
#fig.savefig('CRU Temperature'+start_year[0:4]+'-'+end_year[0:4]+' '+t_season+' TS.png')

#fig.set_size_inches(13,7)

#fig.set_dpi=1200.0
#plt.savefig('Moisture_Paper_Figure_mr'+t_season+'_1970.png',bbox_inches='tight')
plt.show()

In [ ]:
anom_series = t2m_months - t2m_clim_months_avg
extent = anom_series.sel(lat=np.arange(ext_s,ext_n,step=.5))
extent = extent.sel(lon=np.arange(ext_w+360,ext_e+360,step=.5))
extent_series = extent.mean(dim=['lat','lon'])
slope, intercept, r_value, p_value, std_err = stats.linregress(np.arange(0,len(extent_series)),extent_series.values)
line = slope*np.arange(0,len(extent_series.values))+intercept



fig, ax = plt.subplots();
plt.scatter(extent_series['year'],extent_series.values, c=extent_series.values,vmin=-3,vmax=2,cmap='RdBu_r',s=30);
#trend = ax.plot(anom['time'],line)
base = np.zeros(len(extent_series.values))
ax.plot(extent_series['year'],base,c='grey')
ax.set_title('North America 2m Temperature Anomaly '+t_season+' 1970-2010')
ax.set_xlabel('Year')
ax.set_ylabel('Temp Anomaly [C]')
trend = ax.plot(extent_series['year'],line)
#fig.savefig('2m_temp_anom_'+t_season+'_na_sp_1970.png')

In [ ]:
anom_series

In [ ]:
import matplotlib
matplotlib.matplotlib_fname()